# 🧼 Seed Data Cleaning Script
This script automates the cleaning and preparation of product seed data. It removes HTML tags, filters out explicit content, and reorganizes rows based on user-defined keywords to streamline annotation workflows.

In [1]:
import pandas as pd
import re


### 🔧 User-defined Settings

In [2]:
# === USER-DEFINED SETTINGS ===
# Customize these lists as needed

INPUT_CSV = "sample_input.csv"
OUTPUT_XLSX = "cleaned_output.xlsx"
EXPLICIT_WORDS = ["badword", "offensive", "explicit"]  # Customize as needed
SORT_KEYWORDS = ["shirt", "jacket"]  # Reorder priority


### 🧼 Cleaning Functions

In [3]:

# === LOAD DATA ===
df = pd.read_csv(INPUT_CSV)

# === Track rejected rows ===
rejected_unreadable = pd.DataFrame()
rejected_explicit = pd.DataFrame()
rejected_duplicates = pd.DataFrame()

# === 1. REMOVE UNREADABLE WPIDs ===
# WPIDs should be alphanumeric and > 6 characters
mask_valid_wpid = df['WPID'].astype(str).str.fullmatch(r'[A-Za-z0-9]{7,}')
rejected_unreadable = df[~mask_valid_wpid]
df = df[mask_valid_wpid]

# === 2. REMOVE EXPLICIT CONTENT ===
def contains_explicit(text):
    if pd.isnull(text):
        return False
    text = text.lower()
    return any(word in text for word in EXPLICIT_WORDS)

mask_explicit = df['Product Name'].apply(contains_explicit)
rejected_explicit = df[mask_explicit]
df = df[~mask_explicit]

# === 3. REMOVE DUPLICATES (Keep First) ===
df_before = df.copy()
df = df.drop_duplicates()
rejected_duplicates = df_before[~df_before.index.isin(df.index)]

# === 4. CLEAN URL QUOTES ===
for col in ['Image URL1', 'Image URL2', 'Image URL3']:
    df[col] = df[col].astype(str).str.replace('"', '', regex=False)

# === 5. REARRANGE BY KEYWORDS ===
def sort_priority(name):
    if pd.isnull(name):
        return (len(SORT_KEYWORDS), name)
    name_lower = name.lower()
    for i, kw in enumerate(SORT_KEYWORDS):
        if kw.lower() in name_lower:
            return (i, name)
    return (len(SORT_KEYWORDS), name)

df['__sort_order__'] = df['Product Name'].apply(sort_priority)
df = df.sort_values('__sort_order__').drop(columns='__sort_order__')

# === 6. REORDER COLUMNS TO SCHEMA ===
desired_order = ['WPID', 'Item ID', 'Product Name', 'Image URL1', 'Image URL2', 'Image URL3']
df = df[[col for col in desired_order if col in df.columns]]

# === 7. SAVE OUTPUT TO MULTI-SHEET EXCEL ===
with pd.ExcelWriter(OUTPUT_XLSX) as writer:
    df.to_excel(writer, sheet_name="Cleaned_Data", index=False)
    rejected_unreadable.to_excel(writer, sheet_name="Unreadable_WPIDs", index=False)
    rejected_explicit.to_excel(writer, sheet_name="Explicit_Content", index=False)
    rejected_duplicates.to_excel(writer, sheet_name="Duplicates", index=False)

print("✅ Cleaning complete. Output saved to:", OUTPUT_XLSX)


✅ Cleaning complete. Output saved to: cleaned_output.xlsx
